# Few-shot JERX task offline inference

In [ ]:
#|default_exp jerx.offline.llm

In [ ]:
#|hide
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
#|export
from pathlib import Path
import pandas as pd
from bellek.jerx.utils import parse_triplets
from bellek.logging import get_logger

log = get_logger(__name__)

In [ ]:
#|hide
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# |export

def parse_triplet_response(response: str, *args, **kwargs) -> list[tuple[str, str, str]]:
    triplets = parse_triplets(response.strip())
    return [(e1, rel, e2) if e1 != e2 else (e1, rel, e2 + "(obj)") for e1, rel, e2 in triplets]


def make_kg_triplet_extract_fn(inference_cache_filepath: Path):
    df = pd.read_json(inference_cache_filepath, lines=True, orient="records")
    df.set_index("text", inplace=True)

    def extract_kg_triplets(text: str) -> list[tuple[str, str, str]]:
        return parse_triplet_response(df.loc[text]['generation'])

    return extract_kg_triplets

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()